In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_name = 'EleutherAI/pythia-160m'
model_name = 'EleutherAI/pythia-70m'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
model_og = AutoModelForCausalLM.from_pretrained(model_name).to(device)

In [5]:
!python auto_gen_components.py

/bin/bash: line 1: python: command not found


In [6]:
def get_input_after_tokenizer(inp: str):
		tokenizer = AutoTokenizer.from_pretrained(model_name)
		inp_out = tokenizer(inp, return_tensors="pt")
		print(inp_out)
		inputs = inp_out['input_ids'].squeeze(0)
		# print(inputs, inputs.shape)
		# TODO: IDK ABOUT WHATS GOING ON W/ TOKEN SIZE VS Vocab Size
		# Vocab size is 50204 and inp size is 50304
		one_hot = torch.zeros((inputs.shape[0], 50304), dtype=torch.int)
		for i in range(inputs.shape[0]):
			one_hot[i, inputs[i]] = 1
		# one_hot[inputs['input_ids'][0, 0]] = 1
		return one_hot#, inp_out['attention_mask']
get_input_after_tokenizer('hello world')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], dtype=torch.int32)

In [7]:
inps_one_hot = get_input_after_tokenizer("hello world")
seq_len = inps_one_hot.shape[0]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


{'input_ids': tensor([[25521,  1533]]), 'attention_mask': tensor([[1, 1]])}


In [8]:
model_og.gpt_neox.layers[0]

GPTNeoXLayer(
  (input_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_layernorm): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  (post_attention_dropout): Dropout(p=0.0, inplace=False)
  (post_mlp_dropout): Dropout(p=0.0, inplace=False)
  (attention): GPTNeoXAttention(
    (rotary_emb): GPTNeoXRotaryEmbedding()
    (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
    (dense): Linear(in_features=512, out_features=512, bias=True)
    (attention_dropout): Dropout(p=0.0, inplace=False)
  )
  (mlp): GPTNeoXMLP(
    (dense_h_to_4h): Linear(in_features=512, out_features=2048, bias=True)
    (dense_4h_to_h): Linear(in_features=2048, out_features=512, bias=True)
    (act): GELUActivation()
  )
)

In [9]:
from simplified import ModelSel

model_sel = ModelSel(model_og)
model_sel

ModelSel(
  (embed_linear): Linear(in_features=50304, out_features=512, bias=False)
  (layer_norm): SimplfiedLayerNorm(
    (eps): Linear(in_features=512, out_features=512, bias=True)
    (ones_linear): Linear(in_features=512, out_features=512, bias=False)
    (ones_linear_neg): Linear(in_features=512, out_features=512, bias=False)
  )
  (attn): FixedAttentionMask(
    (attn): GPTNeoXAttention(
      (rotary_emb): GPTNeoXRotaryEmbedding()
      (query_key_value): Linear(in_features=512, out_features=1536, bias=True)
      (dense): Linear(in_features=512, out_features=512, bias=True)
      (attention_dropout): Dropout(p=0.0, inplace=False)
    )
    (slice_query): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_value): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_key): Slicer(
      (slicer): Linear(in_features=192, out_features=64, bias=True)
    )
    (slice_rotary): Slicer(
      (slicer): Li

In [10]:
inps_one_hot_formatted = inps_one_hot.float()
# embed_linear.forward(inps_one_hot).shape
model_sel(inps_one_hot_formatted).shape

torch.Size([2, 1536])
QKV SIZE torch.Size([2, 1536]) torch.Size([2])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size(

torch.Size([2, 512])

In [11]:
inps_one_hot_formatted = inps_one_hot.float()
torch.onnx.export(model_sel, inps_one_hot_formatted,
                  'model_sel.onnx')	

torch.Size([2, 1536])
QKV SIZE torch.Size([2, 1536]) torch.Size([2])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])


/home/lev/.local/lib/python3.10/site-packages/transformers/models/gpt_neox/modeling_gpt_neox.py:557: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if seq_len > self.max_seq_len_cached:
/home/lev/code/research/dictator_what/maraboupy_usage/tmp/attn_head.py:48: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if key_length > self.bias.shape[-1]:
/home/lev/code/research/dictator_what/maraboupy_usage/tmp/attn_head.py:78: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to

ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])


ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])
torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2, 16]) torch.Size([2])
ATTN OUTPUT torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64]) torch.Size([2, 64])


## Load Maraboupy

In [13]:
# TODO: maybe simplify the model back....
!pip install git+https://github.com/NeuralNetworkVerification/Marabou.git

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/NeuralNetworkVerification/Marabou.git to /tmp/pip-req-build-293e1crc
  Running command git clone --filter=blob:none --quiet https://github.com/NeuralNetworkVerification/Marabou.git /tmp/pip-req-build-293e1crc
  Resolved https://github.com/NeuralNetworkVerification/Marabou.git to commit 8854d50af873364ef0c0ae61bc1d7144f9e9c519
  Installing build dependencies ... 

In [2]:
!export PYTHONPATH="$PYTHONPATH:/home/lev/code/research/ai/dictator/Marabou"
import sys
import numpy as np

## %
# Path to Marabou folder if you did not export it
sys.path.append('/home/lev/code/research/ai/dictator/Marabou')
from maraboupy import Marabou

2024-03-15 17:31:37.148975: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-15 17:31:37.149072: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-15 17:31:37.154474: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
!pwd

In [12]:
options = Marabou.createOptions(verbosity = 1)
print("Simple Attention Head")
filename = "model_sel.onnx"
network = Marabou.read_onnx(filename)#, inputNames=inputNames, outputNames=[outputName])

Simple Attention Head


TypeError: can only concatenate tuple (not "list") to tuple